In [1]:
using Revise
using JuliaBUGS: CollectVariables, program!, @bugsast

[ Info: Precompiling JuliaBUGS [d71a52e4-b6e4-48cf-ab6f-7c78c729c779]


In [2]:
model_def = @bugsast begin
    a ~ dnorm(0, 1)
    b ~ dnorm(0, a)
    for i in 1:N
        c[i] ~ dnorm(a, b)
    end
end

quote
    $(Expr(:~, :a, :(dnorm(0, 1))))
    $(Expr(:~, :b, :(dnorm(0, a))))
    for i = 1:N
        $(Expr(:~, :(c[i]), :(dnorm(a, b))))
    end
end

In [3]:
data = Dict(:N => 3)

Dict{Symbol, Int64} with 1 entry:
  :N => 3

The first pass simply tries to collect all the variables in the model. 
In a correctly written BUGS model, a variable need to appear either on the LHS of an assignment, or in the data. 
So, the first pass simply look at the LHS of all the possible assignments. 
In the case where LHS is a symbol, the symbol is treated as a scalar; otherwise, the variable is an element of some array. 
In the latter case, the index will be resolved or error will be reported.

In [4]:
vars, arrays_map = program!(CollectVariables(), model_def, data)

(Vars(c[3] => 5, c[1] => 3, c[:] => 6, a => 1, c[2] => 4, b => 2), Dict{Any, Any}(:c => [3, 4, 5]))

In [5]:
vars # five variables

Vars(c[3] => 5, c[1] => 3, c[:] => 6, a => 1, c[2] => 4, b => 2)

In [6]:
arrays_map # store the array whose elements are the variables ids

Dict{Any, Any} with 1 entry:
  :c => [3, 4, 5]

The second pass will try to figure out the dependency between all the variables. 
The dependency is represented as a DAG.

In [7]:
dep_graph = program(DependencyGraph(), model_def, data, vars, arrays_map)

{5, 7} directed simple Int64 graph

In [8]:
using Graphs; adjacency_matrix(dep_graph)

5×5 SparseArrays.SparseMatrixCSC{Int64, Int64} with 7 stored entries:
 ⋅  1  1  1  1
 ⋅  ⋅  1  1  1
 ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅

## Some thoughts and next steps

* The first pass will figure out the size of arrays.
* The second pass is necessary if the target is translating BUGS program to Turing, as order is critical. In case which the target is LogDensityProblems, we will need dependency to figure out the order of logical assignment and compute Gibbs-style logdensity.
* I am still working on how to translate BUGS program to LogDensityProblems automatically. For now, the biggest problem is dealing with logical assignments. Figuring out orders of logical assignments is not trivial, but with dependency graph, it should be doable. The other issue is that autodiff and mutation is not compatible; so if there exist logical assignments to array elements, the direct way of translation will not work.
* SymbolicPPL's graph representation should be relatively easy to convert to a LogDensityProblem.